In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import ast
import re
import sys
print('sys.executable =', sys.executable)
import importlib
print('keybert spec =', importlib.util.find_spec('keybert'))
import seaborn as sns
from keybert import KeyBERT
from nltk.corpus import stopwords

# Load dataset
df = pd.read_csv("Uitgebreide_VKM_dataset.csv")

sys.executable = C:\Users\jeroe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
keybert spec = ModuleSpec(name='keybert', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001D8E2976A90>, origin='C:\\Users\\jeroe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\keybert\\__init__.py', submodule_search_locations=['C:\\Users\\jeroe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\keybert'])


C:\Users\jeroe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df.columns = df.columns.str.strip()
print(df.columns)
df_copy = df.copy()

Index(['id', 'name', 'shortdescription', 'description', 'content',
       'studycredit', 'location', 'contact_id', 'level', 'learningoutcomes',
       'Rood', 'Groen', 'Blauw', 'Geel', 'module_tags',
       'interests_match_score', 'popularity_score', 'estimated_difficulty',
       'available_spots', 'start_date'],
      dtype='object')


zorg 22
innovatie 18
welzijn 15
onderzoek 14
design 12
interdisciplinair 12
data 12
storytelling 11
management 10
industry 10
software 10
smart 9
gedrag 8
jongeren 7
gezondheid 6
health 6
ondernemen 6
ontwerpen 6
art 6
biobased 6
duurzaamheid 6
creatief 5
kwaliteitsmanagement 5
kunst 5
samenleving 5
infrastructuur 5
ethiek 5
chemie 5
palliatieve 4
redeneren 4
duurzaam 4
pedagogische 4
technologie 4
jeugd 4
handelen 4
sociale 4
maatschappelijk 4
prototyping 4
virtual 4
techniek 4
advisering 4
development 4
robotisering 4
ai 4
robots 4
forensisch 4
scenario 4
organische 4
trauma 3
gemeenschappen 3


In [8]:
# Basislijst Nederlandse stopwoorden (kan je uitbreiden)

DUTCH_STOPWORDS = set(stopwords.words('dutch'))

TAG_STOPWORDS = set(stopwords.words("dutch")) | {
    "bij", "voor", "met", "door", "zonder", "over",
    "doelgroep", "werk", "werken", "proces", "praktijk",
    "ontwikkeling", "ontwikkelen", "gaan",
    "leren", "school", "module", "thema",
    "student", "opleiding", "kun" , "vanuit",
    "eigen", "zelf", "samen", "samenwerken",
    "jaar", "week", "periode", "naasten", "daarnaast",
    "minor", "studenten", "programma", "keuzemodule",
    "casus", "casussen", "cases", "vraagstukken",
    "stage", "stageschool", "kennismakingsstage",
    "kennis", "vaardigheid", "vaardigheden", "ervaring", "ervaringen",
    "lessen", "onderwerpen", "theorie", "praktijk", "praktische", "inhoudelijke",
    "mee", "doe", "vinden", "vind", "kies", "openstaan",
    "belangrijk", "positief", "mogelijkheden","mogelijkheid",
    "impact", "betekenis", "betekent", "betekenen",
    "you", "your", "are", "will", "what", "then", "like", "choose",
    "interested", "experiencing", "hbo", "and", "the", "persoonlijke",
    "learning", "denken", "maken", "business", "verdieping", "emgeving",
    "bouwen", "thinking", "branding", "maken", "urban", "veiligheid",
    "nieuwe", "test", "gebouwde", "concept", "project", "omgeving", "actuele", "acute",
    "yellow", "belt", "serious", "hrm", "mensen", "snel", "binnen", "materialen", 
    "active", "druk", "context", "leven", "complexe" , "brede", "for", "jouw"
}

def remove_color_columns(df, color_cols=None):
    """Verwijder de kolommen met kleuren als ze bestaan."""
    if color_cols is None:
        color_cols = ['Rood', 'Geel', 'Blauw', 'Groen']

    cols_to_drop = [col for col in color_cols if col in df.columns]
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
    return df

def remove_rows_by_id(df, ids_to_remove):
    """Verwijder rijen op basis van id-lijst."""
    return df[~df['id'].isin(ids_to_remove)]

def _update_short_description_row(row):
    """Row-helper voor shortdescription."""
    desc = str(row.get('description', '')).strip()
    cont = str(row.get('content', '')).strip()
    short = str(row.get('shortdescription', '')).strip()

    # Only change when short_description == 'Ntb' of 'nan'
    if short.lower() == 'ntb' or short.lower() == 'nan':
        # If description or content is also 'ntb', keep 'Ntb'
        if desc.lower() == 'ntb' or cont.lower() == 'ntb':
            return 'Ntb'
        else:
            return f"{desc} {cont}".strip()
    return short

def update_short_description_column(df):
    """Pas de shortdescription-kolom aan met de logica uit _update_short_description_row."""
    df['shortdescription'] = df.apply(_update_short_description_row, axis=1)
    return df

def fill_missing_descriptions_and_contents(df,
                              desc_col="description",
                              cont_col="content",
                              short_col="shortdescription"):
    """
    Vervangt lege of NTB-achtige description/content door shortdescription.
    """
    
    # Alle varianten die betekenen: "niet bekend", "nog te bepalen", etc.
    NTB_VALUES = {
        "", " ", "ntb", "n.t.b.", "n.t.b", "nnb", "volgt",
        "nader te bepalen", "nog nader te bepalen",
        "nog te bepalen", "nog niet bekend", "none", "nan"
    }

    def is_missing_or_ntb(value):
        if value is None:
            return True
        text = str(value).strip().lower()
        return text in NTB_VALUES

    def fix_row(row):
        short = str(row.get(short_col, "")).strip()
        
        # Alleen vervangen als shortdescription wél bruikbaar is
        if is_missing_or_ntb(short):
            return row  # niets invullen als shortdesc zelf "ntb" is
        
        # Description vervangen?
        if is_missing_or_ntb(row.get(desc_col)):
            row[desc_col] = short
        
        # Content vervangen?
        if is_missing_or_ntb(row.get(cont_col)):
            row[cont_col] = short
        
        return row

    # Pas op elke rij toe
    return df.apply(fix_row, axis=1)

def clean_learningoutcomes_column(df):
    """Schoon de learningoutcomes-kolom op."""
    df['learningoutcomes'] = df['learningoutcomes'].replace(
        {
            r'(?i)^(ntb|n\.t\.b\.|n\.n\.b\.|volgt|nog nader te bepalen|nog te formuleren|nog niet bekend|nog te bepalen|\s*)$':
            'Nader te bepalen'
        },
        regex=True
    )

    df['learningoutcomes'] = df['learningoutcomes'].fillna('Nader te bepalen')
    return df

def set_year_to_2026(date_value):
    """
    Zet de datum om naar jaar 2026, behoudt maand en dag.
    """
    try:
        d = pd.to_datetime(date_value)
        return d.replace(year=2026)
    except Exception:
        return date_value  # fallback als datum niet geldig is

def parse_tags(value):
    """Zorg dat tags altijd een Python-lijst worden."""
    if isinstance(value, list):
        return value
    if pd.isna(value):
        return []
    if isinstance(value, str):
        s = value.strip()
        if not s:
            return []
        if s.startswith('[') and s.endswith(']'):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, list):
                    return parsed
            except (ValueError, SyntaxError):
                pass
        return [p.strip() for p in s.split(',') if p.strip()]
    return []

def clean_tag_phrase(tag, stopwords=TAG_STOPWORDS):
    """
    Schoont een tag op:
    - lowercase
    - verwijdert stopwoorden
    - verwijdert dubbele woorden
    - geeft lege string terug als er niks nuttigs overblijft
    """
    if not isinstance(tag, str):
        tag = str(tag)

    tag = tag.lower().strip()
    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", tag)

    cleaned_tokens = []
    seen = set()

    for tok in tokens:
        if tok in stopwords:
            continue
        if tok in seen:
            continue
        seen.add(tok)
        cleaned_tokens.append(tok)

    return " ".join(cleaned_tokens).strip()

def preprocess_text_for_keybert(description, content, stopwords=DUTCH_STOPWORDS):
    """Maak een tekst schoon als input voor KeyBERT."""
    desc = str(description) if description is not None else ""
    cont = str(content) if content is not None else ""
    text = f"{desc} {cont}".lower()

    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)
    filtered = [t for t in tokens if t not in stopwords and len(t) > 2]

    return " ".join(filtered)

def generate_tags_from_text(description, content,
                            kw_model=None,
                            stopwords=DUTCH_STOPWORDS):
    """
    Genereer ruwe tags (phrases) uit description + content m.b.v. KeyBERT.
    - Geen harde limiet op aantal tags; ruis wordt later weggefilterd.
    """
    if kw_model is not None:
        clean_text = preprocess_text_for_keybert(description, content, stopwords)
        if not clean_text.strip():
            return []

        keywords = kw_model.extract_keywords(
            clean_text,
            keyphrase_ngram_range=(1, 2),
            stop_words=list(stopwords),
            top_n=10  # ruim aantal kandidaten, volgorde is op score
        )

        raw_tags = [phrase.strip() for phrase, score in keywords if phrase.strip()]
        return raw_tags

    # Fallback zonder KeyBERT
    desc = str(description) if description is not None else ""
    cont = str(content) if content is not None else ""
    text = f"{desc} {cont}".lower()

    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)
    raw_tags = []
    seen = set()
    for tok in tokens:
        if tok in stopwords:
            continue
        if len(tok) <= 2:
            continue
        if tok in seen:
            continue
        seen.add(tok)
        raw_tags.append(tok)

    return raw_tags

def tags_are_empty_or_ntb(tags):
    """
    True als lijst leeg is of alleen varianten van 'ntb' / 'nog te bepalen' bevat.
    """
    if tags is None:
        return True
    if not isinstance(tags, list):
        tags = [tags]

    normalized = [str(t).strip().lower() for t in tags if str(t).strip()]
    if not normalized:
        return True

    NTB_VALUES = {"ntb", "n.t.b", "n.t.b.", "nog te bepalen"}
    return all(t in NTB_VALUES for t in normalized)

def process_module_tags_row(row,
                            tag_col='module_tags',
                            kw_model=None):
    """
    Centrale tag-logica:
    - parse bestaande tags
    - als leeg/NTB -> nieuwe tags uit description+content (KeyBERT)
    - ALLE tags door clean_tag_phrase (met TAG_STOPWORDS)
    - duplicaten eruit
    """
    existing_tags = parse_tags(row.get(tag_col, []))

    if tags_are_empty_or_ntb(existing_tags):
        candidate_tags = generate_tags_from_text(
            row.get('description', ''),
            row.get('content', ''),
            kw_model=kw_model,
            stopwords=DUTCH_STOPWORDS,
        )
    else:
        candidate_tags = existing_tags

    cleaned_tags = []
    seen = set()
    for tag in candidate_tags:
        cleaned = clean_tag_phrase(tag, stopwords=TAG_STOPWORDS)
        if not cleaned:
            continue
        if cleaned in seen:
            continue
        seen.add(cleaned)
        cleaned_tags.append(cleaned)

    return cleaned_tags

def clean_and_fill_module_tags(df,
                               tag_col='module_tags',
                               kw_model=None):
    """Past process_module_tags_row toe op elke rij om module_tags op te schonen/aan te vullen."""
    df = df.copy()
    if tag_col not in df.columns:
        return df

    df[tag_col] = df.apply(
        lambda row: process_module_tags_row(
            row,
            tag_col=tag_col,
            kw_model=kw_model
        ),
        axis=1
    )
    return df

def clean_df(df, kw_model=None):
    """Pipeline: voert alle cleanup-stappen uit op een kopie van de dataframe."""
    df_copy = df.copy()

    df_copy = remove_color_columns(df_copy)
    df_copy = remove_rows_by_id(df_copy, ids_to_remove=[293, 294, 295, 232])
    df_copy = update_short_description_column(df_copy)
    df_copy = clean_learningoutcomes_column(df_copy)
    df_copy = fill_missing_descriptions_and_contents(df_copy)

    if 'start_date' in df_copy.columns:
        df_copy['start_date'] = df_copy['start_date'].apply(set_year_to_2026)

    df_copy = clean_and_fill_module_tags(
        df_copy,
        tag_col='module_tags',   # zorg dat je kolom zo heet
        kw_model=kw_model
    )

    return df_copy

# 1) maak eerst je KeyBERT-model
kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

# 2) run de cleaning + taggeneratie
df_copy = clean_df(df, kw_model=kw_model)

df_copy.to_csv("Uitgebreide_VKM_dataset_cleaned2.csv", index=False)
display(df_copy)

,id,name,shortdescription,description,content,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,Kennismaking met Psychologie,"Brein, gedragsbeinvloeding, ontwikkelingspsych...",In deze module leer je hoe je gedrag van jezel...,In deze module leer je hoe je gedrag van jezel...,15,Den Bosch,58,NLQF5,A. Je beantwoordt vragen in een meerkeuze kenn...,"[brein, gedragsbeinvloeding, ontwikkelingspsyc...",0.54,319,1,79,2026-12-24
1,160,Learning and working abroad,"Internationaal, persoonlijke ontwikkeling, ver...",Studenten kiezen binnen de (stam) van de oplei...,Studenten kiezen binnen de (stam) van de oplei...,15,Den Bosch,58,NLQF5,De student toont professioneel gedrag conform ...,"[internationaal, verpleegkunde]",0.92,172,5,56,2026-12-20
2,161,Proactieve zorgplanning,"Proactieve zorgplanning, cocreatie, ziekenhuis",Het Jeroen Bosch ziekenhuis wil graag samen me...,Het Jeroen Bosch ziekenhuis wil graag samen me...,15,Den Bosch,59,NLQF5,De student past pro actieve zorgplanning toe b...,"[proactieve, zorgplanning, cocreatie, ziekenhuis]",0.78,217,5,55,2026-09-23
3,162,Rouw en verlies,"Rouw & verlies, palliatieve zorg & redeneren, ...",In deze module wordt stil gestaan bij rouw en ...,In deze module wordt stil gestaan bij rouw en ...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"[rouw, verlies, palliatieve, zorg, redeneren, ...",0.69,454,1,54,2026-10-25
4,163,Acuut complexe zorg,"Acute zorg, complexiteit, ziekenhuis, revalidatie",In deze module kunnen studenten zich verdiepen...,In deze module kunnen studenten zich verdiepen...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"[zorg, complexiteit, ziekenhuis, revalidatie]",0.40,178,5,38,2026-11-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,393,Ethiek & kritisch denken,"ethiek, filosofie, kritisch denken, ethisch ha...",Ethiek en ethisch handelen wordt steeds belang...,Ethiek en ethisch handelen wordt steeds belang...,30,Breda en Den Bosch,124,NLQF6,De student demonstreert persoonlijke groei op ...,"[ethiek, filosofie, kritisch, ethisch, handele...",0.37,248,5,32,2026-10-17
207,394,Avans Innovative Studio Junior,"persoonlijke ontwikkeling, interdisciplinair, ...",Je leert hoe welke waarde jouw werk heeft. Hoe...,Je leert hoe welke waarde jouw werk heeft. Hoe...,15,Breda,92,NLQF6,De student demonstreert persoonlijke groei op ...,"[interdisciplinair, innovatie, prototyping, va...",0.73,299,5,62,2026-09-01
208,395,Avans Innovative Studio Senior,Ook heb je de mogelijkheid om met creatieve on...,Ook heb je de mogelijkheid om met creatieve on...,Ook heb je de mogelijkheid om met creatieve on...,30,Breda,92,NLQF6,De student demonstreert persoonlijke groei op ...,"[creatieve ondernemers, creatieve, onderwijsmu...",0.30,105,5,62,2026-11-29
209,396,Succesvol managen van een project,"Project Evaluatie, Scrum, Planning, Agile, Pri...","Leren om succesvol een project (innovatie, cha...","Leren om succesvol een project (innovatie, cha...",15,Breda,125,NLQF5,"1) Inzicht/kennis in de diverse factoren, zoal...","[evaluatie, scrum, planning, agile, prince2, m...",0.40,135,5,67,2026-09-14


In [10]:
from collections import Counter

def get_tag_token_frequencies(df, tag_col="module_tags"):
    # module_tags moeten al lijsten zijn
    all_tokens = []

    for tags in df[tag_col]:
        for tag in tags:
            tag = str(tag).lower()
            tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", tag)
            all_tokens.extend(tokens)

    return Counter(all_tokens)

freqs = get_tag_token_frequencies(df_copy, tag_col="module_tags")

# Laat de 50 meest voorkomende tokens zien
for word, count in freqs.most_common(50):
    print(word, count)

zorg 22
innovatie 18
welzijn 15
onderzoek 14
design 12
interdisciplinair 12
data 12
storytelling 11
management 10
industry 10
software 10
smart 9
gedrag 8
jongeren 7
gezondheid 6
health 6
ondernemen 6
ontwerpen 6
art 6
biobased 6
duurzaamheid 6
creatief 5
kwaliteitsmanagement 5
kunst 5
samenleving 5
infrastructuur 5
ethiek 5
chemie 5
palliatieve 4
redeneren 4
duurzaam 4
pedagogische 4
technologie 4
jeugd 4
handelen 4
sociale 4
maatschappelijk 4
prototyping 4
virtual 4
techniek 4
advisering 4
development 4
robotisering 4
ai 4
robots 4
forensisch 4
scenario 4
organische 4
trauma 3
gemeenschappen 3
